# Entrenamiento del meta-modelo

Aqué entrenaremos un modelo XGBoosting 

Como tenemos los datos un poco desordenados, primero los tengo que organizar, ya que para el entrenamiento necesito los datos con los que se entreno el PPO, los labeles obtenidos en el notebook `00_meta_labeling`, y un indicador adicional de regímenes que lo sacamos mediante cadenas de markov en el notebook `01_hmm`.

Voy a cargar los datos in-sample y out-of-sample, y los datos de los labels.

In [1]:
import sys
import os
import pandas as pd

# Añadir el directorio raíz del proyecto al PYTHONPATH
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt

In [7]:
SPY_ORIG_IN_SAMPLE = pd.read_parquet(r'C:\Users\adelapuente\Desktop\math_tfm\02_training_data\SPY_original_processed_in_sample.parquet')
SPY_ORIG_OO_SAMPLE = pd.read_parquet(r'C:\Users\adelapuente\Desktop\math_tfm\02_training_data\SPY_original_processed_out_of_sample.parquet')

SPY_VOLUME_IN_SAMPLE = pd.read_parquet(r'C:\Users\adelapuente\Desktop\math_tfm\02_training_data\SPY_volume_processed_in_sample.parquet')
SPY_VOLUME_OO_SAMPLE = pd.read_parquet(r'C:\Users\adelapuente\Desktop\math_tfm\02_training_data\SPY_volume_processed_out_of_sample.parquet')

SPY_DOLLAR_IN_SAMPLE = pd.read_parquet(r'C:\Users\adelapuente\Desktop\math_tfm\02_training_data\SPY_dollar_processed_in_sample.parquet')
SPY_DOLLAR_OO_SAMPLE = pd.read_parquet(r'C:\Users\adelapuente\Desktop\math_tfm\02_training_data\SPY_dollar_processed_out_of_sample.parquet')

In [8]:
SPY_ORIG_IN_SAMPLE

,date,feature_open,feature_high,feature_low,feature_close,feature_SMA_20,feature_SMA_50,feature_EMA_14,feature_BBL_20_2.0,feature_BBM_20_2.0,feature_BBU_20_2.0,feature_BBB_20_2.0,feature_BBP_20_2.0,feature_RSI,feature_MACD,feature_MACD_Signal,feature_MACD_Diff,feature_ATR,feature_CCI,close
50,2018-01-02 10:20:00,0.105565,0.105166,0.105508,0.105134,0.102575,0.099922,0.102590,0.113299,0.102575,0.099683,0.002871,0.613177,0.641031,0.596034,0.600681,0.666380,0.047352,0.567844,242.415
51,2018-01-02 10:21:00,0.105565,0.105221,0.105508,0.105134,0.102585,0.099955,0.102605,0.113306,0.102585,0.099694,0.002895,0.602615,0.641031,0.595606,0.600352,0.666096,0.047228,0.570087,242.415
52,2018-01-02 10:22:00,0.105553,0.105166,0.105482,0.105186,0.102596,0.099985,0.102625,0.113313,0.102596,0.099711,0.002969,0.641720,0.657091,0.595326,0.600024,0.666101,0.047029,0.559420,242.433
53,2018-01-02 10:23:00,0.105605,0.105166,0.105456,0.105160,0.102609,0.100022,0.102638,0.113332,0.102609,0.099718,0.002888,0.604236,0.641634,0.594939,0.599671,0.665936,0.046921,0.546190,242.424
54,2018-01-02 10:24:00,0.105591,0.105128,0.105429,0.105119,0.102626,0.100063,0.102645,0.113386,0.102626,0.099697,0.002384,0.553194,0.617308,0.594431,0.599270,0.665612,0.046787,0.522106,242.410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611519,2023-12-29 15:56:00,0.780934,0.781778,0.781592,0.781622,0.782556,0.782878,0.782228,0.784702,0.782556,0.780874,0.009770,0.384863,0.435451,0.573100,0.578160,0.659329,0.023627,0.428680,475.120
611520,2023-12-29 15:57:00,0.781544,0.781428,0.781766,0.781215,0.782460,0.782850,0.782129,0.784573,0.782460,0.780813,0.010002,0.339042,0.403404,0.572239,0.576897,0.659771,0.023225,0.422106,474.980
611521,2023-12-29 15:58:00,0.781152,0.781545,0.781868,0.781563,0.782385,0.782814,0.782089,0.784519,0.782385,0.780717,0.009864,0.403597,0.441355,0.572740,0.576004,0.662273,0.022934,0.444547,475.100
611522,2023-12-29 15:59:00,0.781457,0.783232,0.782144,0.782290,0.782351,0.782792,0.782153,0.784530,0.782351,0.780638,0.009558,0.524339,0.511127,0.575394,0.575907,0.667667,0.026650,0.525000,475.350


In [5]:
BTC_ORIG_IN_SAMPLE = pd.read_parquet(r'C:\Users\adelapuente\Desktop\math_tfm\02_training_data\BTC_original_processed_in_sample.parquet')
BTC_ORIG_OO_SAMPLE = pd.read_parquet(r'C:\Users\adelapuente\Desktop\math_tfm\02_training_data\BTC_original_processed_out_of_sample.parquet')

BTC_VOLUME_IN_SAMPLE = pd.read_parquet(r'C:\Users\adelapuente\Desktop\math_tfm\02_training_data\BTC_volume_processed_in_sample.parquet')
BTC_VOLUME_OO_SAMPLE = pd.read_parquet(r'C:\Users\adelapuente\Desktop\math_tfm\02_training_data\BTC_volume_processed_out_of_sample.parquet')

BTC_DOLLAR_IN_SAMPLE = pd.read_parquet(r'C:\Users\adelapuente\Desktop\math_tfm\02_training_data\BTC_dollar_processed_in_sample.parquet')
BTC_DOLLAR_OO_SAMPLE = pd.read_parquet(r'C:\Users\adelapuente\Desktop\math_tfm\02_training_data\BTC_dollar_processed_out_of_sample.parquet')